In [108]:

import pandas as pd
from bcb import sgs, Expectativas
from datetime import datetime, timedelta
import numpy as np

In [109]:
em = Expectativas()
# em.describe()
ep_m_mensal = em.get_endpoint('ExpectativaMercadoMensais')

In [110]:
# função para fazer a formatação das DataFrames
def formata_df(df: pd.DataFrame):
    # Formatando as datas
    df['DataReferencia'] = pd.to_datetime(df['DataReferencia'], format='%m/%Y')

    # Colocando DataReferencia como header
    df_pivot = df.pivot_table(index='Data', columns='DataReferencia', values='Mediana')
    
    # Formatando data e ordem da DataReferencia
    df_pivot = df_pivot.sort_index(axis=1)
    df_pivot.columns = df_pivot.columns.strftime('%m/%Y')
    return df_pivot.applymap(lambda x: '{:.4f}'.format(x) if not pd.isnull(x) else np.nan)



In [111]:
# IPCA mensal
ipca_mensal = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA', ep_m_mensal.Data >= '2023-01-01').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_mensal_pivot = formata_df(ipca_mensal)
# ipca_mensal_pivot

In [112]:
# IPCA Livre
ipca_livre_expec = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA Livres', ep_m_mensal.Data >= '2023-01-01').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_livre_pivot = formata_df(ipca_livre_expec)
# ipca_livre_pivot

In [113]:
# IPCA Adm
ipca_adm_expec = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA Administrados', ep_m_mensal.Data >= '2023-01-01').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_adm_pivot = formata_df(ipca_adm_expec)
# ipca_adm_pivot

In [114]:
# expec IPCA mensal
with pd.ExcelWriter(r'SEU_CAMINHO') as writer:
   ipca_mensal_pivot.to_excel(writer, sheet_name='IPCA Mensal')
   ipca_livre_pivot.to_excel(writer, sheet_name='IPCA Livre')
   ipca_adm_pivot.to_excel(writer, sheet_name='IPCA ADM')
